# Создание мастер-датасета

В этом ноутбуке загружаются 15 таблиц, содержащих социально-демографические, экономические и медицинские характеристики российских регионов.  
Каждая из таблиц предварительно прошла этап EDA.  

В рамках данного ноутбука выполняются следующие шаги:
- загрузка финальных таблиц после очистки;
- фильтрация по годам (2015–2020);
- проверка и согласование названий регионов и структуры данных;
- частичная агрегация, если необходимо;
- объединение всех таблиц в единую мастер-таблицу;
- сохранение финального мастер-датасета для последующего анализа.


In [32]:
import pandas as pd

# Путь к папке с финальными версиями
data_path = "../../data/clean/"

# Загрузка
df_poverty = pd.read_csv(data_path + "poverty_AfterEDA.csv")
df_income = pd.read_csv(data_path + "income_AfterEDA.csv")
df_socdem = pd.read_csv(data_path + "socdem_AfterEDA.csv")
df_drug_alco = pd.read_csv(data_path + "drug_alco_AfterEDA.csv")
df_morbidity = pd.read_csv(data_path + "morbidity_AfterEDA.csv")
df_newborns = pd.read_csv(data_path + "newborns_AfterEDA.csv")
df_housing = pd.read_csv(data_path + "housing_AfterEDA.csv")
df_gross = pd.read_csv(data_path + "gross_AfterEDA.csv")
df_population = pd.read_csv(data_path + "population_AfterEDA.csv")
df_disabled = pd.read_csv(data_path + "disabled_AfterEDA.csv")
df_child_mortality_rural = pd.read_csv(data_path + "child_rural_AfterEDA.csv")
df_child_mortality_urban = pd.read_csv(data_path + "child_urban_AfterEDA.csv")
df_retail = pd.read_csv(data_path + "retail_AfterEDA.csv")
df_welfare = pd.read_csv(data_path + "welfare_AfterEDA.csv")
df_regional_production = pd.read_csv(data_path + "reg_production_AfterEDA.csv")


In [33]:
# Ищет колонку с годами и оставляет нужный диапазон
def filter_years(df, years=None):
    # Все возможные названия колонки года
    possible_cols = ["year", "год", "Год"]
    
    col = None
    for c in possible_cols:
        if c in df.columns:
            col = c
            break

    if col is None:
        raise ValueError("Не найден столбец 'year', 'год' или 'Год' в таблице.")
    
    # Если аргумент years не передан, берем 2015–2020
    if years is None:
        years = list(range(2015, 2021))
    
    return df[df[col].isin(years)]


# Применяем фильтрацию к таблицам, в которых есть год
df_poverty = filter_years(df_poverty)
df_income = filter_years(df_income)
df_socdem = filter_years(df_socdem)
df_drug_alco = filter_years(df_drug_alco)
df_gross = filter_years(df_gross)
df_population = filter_years(df_population)
df_child_mortality_rural = filter_years(df_child_mortality_rural)
df_child_mortality_urban = filter_years(df_child_mortality_urban)
df_retail = filter_years(df_retail)
df_welfare = filter_years(df_welfare)
df_regional_production = filter_years(df_regional_production)
df_newborns = filter_years(df_newborns)
df_disabled = filter_years(df_disabled)

In [34]:
# Оставляем только 2015 и 2016
morbidity_agg = (
    df_morbidity
    [df_morbidity["year"].isin([2015, 2016])]
    .groupby(["region_standard", "year"])["cases"]
    .sum()
    .reset_index()
)

In [35]:
# Преобразуем в широкий формат
drug_alco_pivot = (
    df_drug_alco
    .pivot_table(
        index=["region_standard", "year"],
        columns="diagnosis",
        values="value"
    )
    .reset_index()
)

# Переименуем колонки
drug_alco_pivot.rename(columns={
    "alcohol": "alcohol_rate",
    "drugs": "drugs_rate"
}, inplace=True)


In [36]:
# Агрегируем объём производства по региону и году
regional_production_agg = (
    df_regional_production
    .groupby(["region_standard", "year"])["value"]
    .sum()
    .reset_index()
    .rename(columns={"value": "production_total"})
)

In [37]:
# Словарь с таблицами
tables = {
    "poverty": df_poverty,
    "income": df_income,
    "socdem": df_socdem,
    "drug_alco": drug_alco_pivot,
    "newborns": df_newborns,
    "disabled": df_disabled,
    "morbidity": morbidity_agg,
    "housing": df_housing,
    "workers": df_gross,
    "population": df_population,
    "child_mortality_rural": df_child_mortality_rural,
    "child_mortality_urban": df_child_mortality_urban,
    "retail": df_retail,
    "welfare": df_welfare,
    "regional_production": regional_production_agg
}

# Стандартизация ключевых колонок
for name, df in tables.items():
    df.rename(columns={
        "Регион": "region",
        "Год": "year",
        "год": "year",
        "Region": "region",
        "Year": "year"
    }, inplace=True)
    
    # Если есть колонка 'region' — удалим или оставим только region_standard
    if "region" in df.columns and "region_standard" not in df.columns:
        df.rename(columns={"region": "region_standard"}, inplace=True)
    elif "region" in df.columns and "region_standard" in df.columns:
        df.drop(columns=["region"], inplace=True)


In [38]:
# Удаление дубликатов по ключу region_standard + year
for name, df in tables.items():
    before = df.shape[0]
    df.drop_duplicates(subset=["region_standard", "year"], inplace=True)
    after = df.shape[0]
    print(f"{name:<25} — удалено {before - after} дубликатов")


poverty                   — удалено 12 дубликатов
income                    — удалено 0 дубликатов
socdem                    — удалено 8 дубликатов
drug_alco                 — удалено 0 дубликатов
newborns                  — удалено 12 дубликатов
disabled                  — удалено 0 дубликатов
morbidity                 — удалено 0 дубликатов
housing                   — удалено 0 дубликатов
workers                   — удалено 12 дубликатов
population                — удалено 7 дубликатов
child_mortality_rural     — удалено 12 дубликатов
child_mortality_urban     — удалено 12 дубликатов
retail                    — удалено 14 дубликатов
welfare                   — удалено 0 дубликатов
regional_production       — удалено 0 дубликатов


In [39]:
# Начинаем сбор мастер-датасета с таблицы бедности
master_df = df_poverty.copy()

In [40]:
# Доходы
master_df = master_df.merge(df_income, on=["region_standard", "year"], how="left")

In [41]:
# Соцдем
master_df = master_df.merge(df_socdem, on=["region_standard", "year"], how="left")

In [42]:
# Алкоголь и наркотики
master_df = master_df.merge(drug_alco_pivot, on=["region_standard", "year"], how="left")

In [43]:
# Заболеваемость
master_df = master_df.merge(morbidity_agg, on=["region_standard", "year"], how="left")

In [44]:
# Рождаемость
master_df = master_df.merge(df_newborns, on=["region_standard", "year"], how="left")

In [45]:
# Инвалидность
master_df = master_df.merge(df_disabled, on=["region_standard", "year"], how="left")

In [46]:
# ВРП на душу населения
master_df = master_df.merge(df_gross, on=["region_standard", "year"], how="left")

In [47]:
# Население
master_df = master_df.merge(df_population, on=["region_standard", "year"], how="left")

In [48]:
# Детская смертность в сельской местности
master_df = master_df.merge(df_child_mortality_rural, on=["region_standard", "year"], how="left")

In [49]:
# Детская смертность в городах
master_df = master_df.merge(df_child_mortality_urban, on=["region_standard", "year"], how="left")

In [50]:
# Розничная торговля
master_df = master_df.merge(df_retail, on=["region_standard", "year"], how="left")

In [51]:
# Социальное обеспечение
master_df = master_df.merge(df_welfare, on=["region_standard", "year"], how="left")

In [52]:
# Региональное производство
master_df = master_df.merge(regional_production_agg, on=["region_standard", "year"], how="left")

In [53]:
# Переименовываем столбцы
master_df.rename(columns={
    "region_standard": "region",
    "Дети в возрасте до 16 лет": "children_percent",
    "Население старше трудоспособного возраста": "elderly_percent",
    "Население трудоспособного возраста": "working_age_percent",
    "cases": "crime_cases",
    "ВРП_на_душу": "gdp_per_capita",
    "Младенческая_смертность_x": "infant_mortality_rural",
    "Младенческая_смертность_y": "infant_mortality_urban",
    "Оборот_на_душу": "retail_per_capita"
}, inplace=True)


In [54]:
# Только числовые признаки
df_numeric = master_df.select_dtypes(include='number')

# Вычисление доли пропусков
missing_ratio = df_numeric.isnull().mean().sort_values(ascending=False)

# Отображение признаков с пропусками
missing_ratio[missing_ratio > 0]


crime_cases            0.666667
alcohol_rate           0.400000
drugs_rate             0.400000
elderly_percent        0.356863
children_percent       0.356863
working_age_percent    0.356863
18_30                  0.333333
total                  0.333333
31_40                  0.333333
41_50                  0.333333
51_60                  0.333333
60_                    0.333333
nominal_wage           0.062745
real_wage              0.062745
income_per_capita      0.062745
real_income            0.062745
welfare_percent        0.058824
population             0.023529
dtype: float64

In [55]:
# Заполнение медианой по региону
columns_to_fill = [
    "children_percent", "elderly_percent", "working_age_percent",
    "alcohol_rate", "drugs_rate",
    "total", "18_30", "31_40",
    "41_50", "51_60", "60_",
    "nominal_wage", "real_wage", "income_per_capita", "real_income",
    "welfare_percent", "population"
]

for col in columns_to_fill:
    master_df[col] = master_df.groupby("region")[col].transform(lambda x: x.fillna(x.median()))

# Удаление сильно повреждённого признака
master_df.drop(columns=["cases", "crime_cases"], inplace=True, errors="ignore")

# Финальное заполнение оставшихся пропусков общей медианой
master_df.fillna(master_df.median(numeric_only=True), inplace=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keep

In [56]:
# --- Добавляем производные признаки для дальнейшего анализа ---


# Доля иждивенцев: дети + пожилые
master_df["dependent_percent"] = (master_df["children_percent"] + master_df["elderly_percent"])

# Число инвалидов на 1000 человек населения
master_df["disabled_rate_per_1000"] = (master_df["total"] / master_df["population"]) * 1000

# Суммарный уровень алкоголизма и наркомании
master_df["addiction_rate"] = (master_df["alcohol_rate"] + master_df["drugs_rate"])

# Производство на душу населения
master_df["production_per_capita"] = (master_df["production_total"] / master_df["population"])

# Соотношение бедности и расходов на социальную политику
master_df["poverty_welfare_ratio"] = (master_df["poverty_percent"] / master_df["welfare_percent"])

# Рождаемость на 1000 человек
master_df["birth_rate_per_1000"] = master_df["births"] / master_df["population"] * 1000

# Младенческая смертность в селе на 1000 рождений
master_df["infant_mortality_rural_rate"] = master_df["infant_mortality_rural"] / master_df["births"] * 1000

# Младенческая смертность в городе на 1000 рождений
master_df["infant_mortality_urban_rate"] = master_df["infant_mortality_urban"] / master_df["births"] * 1000

# Инвалиды 18–30 лет на 1000 населения
master_df["disabled_18_30_rate"] = master_df["18_30"] / master_df["population"] * 1000

# Инвалиды 31–40 лет на 1000 населения
master_df["disabled_31_40_rate"] = master_df["31_40"] / master_df["population"] * 1000

# Инвалиды 41–50 лет на 1000 населения
master_df["disabled_41_50_rate"] = master_df["41_50"] / master_df["population"] * 1000

# Инвалиды 51–60 лет на 1000 населения
master_df["disabled_51_60_rate"] = master_df["51_60"] / master_df["population"] * 1000

# Инвалиды 60+ лет на 1000 населения
master_df["disabled_60_plus_rate"] = master_df["60_"] / master_df["population"] * 1000


In [57]:
master_df.to_csv("../../data/clean/master_dataset_2015_2020.csv", index=False)